# Get config scale for the config.json

In [184]:
# Imports
import os
import numpy as np
import torch
from scipy.interpolate import RegularGridInterpolator
import matplotlib
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import json

# Constants
# DATA_DIR = 'scannet/scene0710_00'
DATA_DIR = 'scannetpp/7eac902fd5'
COLMAP_DIR = os.path.join(DATA_DIR, 'colmap/sparse/0')
SENSOR_DEPTH_DIR = os.path.join(DATA_DIR, 'depth')
CONFIG_FILENAME = 'config_new.json'

In [199]:
def get_colmap_camera_matrix(colmap_cameras_path):
    with open(colmap_cameras_path, 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if not line.startswith('#')]    # Skip all lines that begin with # (comments)
        # Extract image size
        # 1 SIMPLE_PINHOLE 624 468 600.941 312 234
        image_size = tuple(map(int, lines[0].split(' ')[2:4]))
        # Extract the camera parameters
        camera_params = lines[0].split(' ')[-3:]
        # Extract the focal length and principal point
        f, cx, cy = map(float, camera_params)
        # Construct the camera matrix
        K = np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]])
    return image_size, K

def get_colmap_dict_points(colmap_points_path):
    colmap_points_dict = {}
    with open(colmap_points_path, 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if not line.startswith('#')]    # Skip all lines that begin with # (comments)
        for line in lines:
            line = line.strip().split()
            # 3D point list with one line of data per point:
            # POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
            point_id = int(line[0])
            x = float(line[1])
            y = float(line[2])
            z = float(line[3])
            r = int(line[4])
            g = int(line[5])
            b = int(line[6])
            error = float(line[7])
            track = line[8:]
            for i in range(8, len(line), 2):
                image_id = int(line[i])
                point2d_idx = int(line[i + 1])
                track.append((image_id, point2d_idx))
            colmap_points_dict[point_id] = (x, y, z, r, g, b, error, track)
    return colmap_points_dict

def get_colmap_points(colmap_images_path, image_id):
    # Read the images.txt file which has the following format:
    #   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
    #   POINTS2D[] as (X, Y, POINT3D_ID)
    with open(colmap_images_path) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if not line.startswith('#')]    # Skip all lines that begin with # (comments)
        # Find the line corresponding to the image_id
        points_2d = []
        points_3d_ids = []
        q = None
        t = None
        for idx, line in enumerate(lines):
            if str(line).endswith(str(image_id) + '.JPG'):
                # Read the camera transformation
                q = list(map(float, lines[idx].split(' ')[1:5]))
                t = list(map(float, lines[idx].split(' ')[5:8]))
                # lines[idx+1] contains the POINTS2D[] line
                line_points = lines[idx+1].split(' ')
                # Read three elements at a time
                for i in range(0, len(line_points), 3):
                    x, y, point_3d_id = line_points[i:i+3]
                    if point_3d_id != '-1':
                        points_2d.append((float(x), float(y)))
                        points_3d_ids.append(int(point_3d_id))

    return q, t, points_2d, points_3d_ids

In [200]:
import numpy as np

def quaternion_to_rotation_matrix(q):
    """
    Convert a quaternion into a 3x3 rotation matrix.
    """
    qw, qx, qy, qz = q
    R = np.array([
        [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]
    ])
    return R

def rotation_traslation_to_matrix(R, t):
    """
    Convert a rotation matrix and a translation vector into a 4x4 transformation matrix.
    """
    Tr = np.eye(4)
    Tr[:3, :3] = R
    Tr[:3, 3] = t
    return Tr

def transform_points(points_3d, Tr):
    """
    Transform 3D points from world coordinates to camera coordinates.
    
    Parameters:
    points_3d : ndarray of shape (n, 3)
        The 3D points in world coordinates.
    T : ndarray of shape (4, 4)
        The transformation matrix from world to camera coordinates.
    Returns:
    points_3d_transformed : ndarray of shape (n, 3)
        The transformed 3D points in camera coordinates.
    """
    # Add a column of ones to the points_3d
    points_3d_homogeneous = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    # Transform the points
    points_3d_transformed = (Tr @ points_3d_homogeneous.T).T[:, :3]
    return points_3d_transformed

def compute_depth(points_3d_transformed):
    """
    Compute the depth values from the transformed 3D points.
    
    Parameters:
    points_3d_transformed : ndarray of shape (n, 3)
        The transformed 3D points in camera coordinates.
    
    Returns:
    depths : ndarray of shape (n,)
        The depth values (z-coordinates) of the transformed points.
    """
    return points_3d_transformed[:, 2]

In [201]:
def project_points(K, points_3d):
    """
    Project 3D points onto the image plane using the intrinsic matrix K.
    
    Parameters:
    K : ndarray of shape (3, 3)
        The intrinsic matrix.
    points_3d : ndarray of shape (n, 3)
        The 3D points in camera coordinates.
    
    Returns:
    points_2d : ndarray of shape (n, 2)
        The projected 2D points on the image plane.
    depths : ndarray of shape (n,)
        The depth values (z-coordinates) of the transformed points.
    """
    # Convert 3D points to homogeneous coordinates
    # points_3d_h = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    
    # Project to 2D (homogeneous coordinates)
    points_2d_h = (K @ points_3d.T).T
    
    # Convert to non-homogeneous coordinates
    points_2d = points_2d_h[:, :2] / points_2d_h[:, 2:]
    depths = points_3d[:, 2]  # Extract depth values (z-coordinates)
    
    return points_2d, depths

def filter_points_in_image(points_2d, depths, image_width, image_height):
    """
    Filter points to determine which ones lie within the camera plane.
    
    Parameters:
    points_2d : ndarray of shape (n, 2)
        The projected 2D points on the image plane.
    depths : ndarray of shape (n,)
        The depth values of the points.
    image_width : int
        The width of the image.
    image_height : int
        The height of the image.
    
    Returns:
    mask : ndarray of shape (n,)
        A boolean array indicating which points are within the image boundaries and in front of the camera.
    """
    mask = (
        (points_2d[:, 0] >= 0) & (points_2d[:, 0] < image_width) &  # x coordinates within image width
        (points_2d[:, 1] >= 0) & (points_2d[:, 1] < image_height) &  # y coordinates within image height
        (depths > 0)  # points in front of the camera
    )
    return mask


In [202]:
def convert_depth_completion_scaling_to_m(depth):
    # convert from depth completion scaling to meter, that means map range 0 .. 1 to range 0 .. 16,38m
    return depth * (2 ** 16 - 1) / 4000.

def compute_rmse(prediction, target):
    return torch.sqrt((prediction - target).pow(2).mean())

In [203]:
# barbara_depths = torch.load(os.path.join(DATA_DIR, 'depths.pth'))
# Get camera matrix
image_size, K = get_colmap_camera_matrix(os.path.join(COLMAP_DIR, 'cameras.txt'))
image_width, image_height = image_size
# Get dictionary of points
points_dict = get_colmap_dict_points(os.path.join(COLMAP_DIR, 'points3D.txt'))
ls_colmap_errors = []
ls_colmap_depths = []
ls_sensor_depths = []
ls_sensor_depths_all = []

# Iterate over the depths in the directory
for filename in os.listdir(SENSOR_DEPTH_DIR):
    #filename = os.path.basename(filename).split('.')[0] + '.npy'
    image_id = filename.split('.')[0]
    print('Image: ', image_id)
    # Get points from COLMAP for the same image
    colmap_images_path = os.path.join(COLMAP_DIR, 'images.txt')
    q, t, _, points_3d_ids = get_colmap_points(colmap_images_path, image_id)
    points_3d = [points_dict[point_id] for point_id in points_3d_ids]
    # Select the first three elements of the points_3d
    points_3d_xyz = np.array([point[:3] for point in points_3d])
    points_error = np.array([point[6] for point in points_3d])
    # Transform the 3D points into camera coordinates
    Tr = rotation_traslation_to_matrix(quaternion_to_rotation_matrix(q), t)
    if points_3d_xyz.shape[0] == 0:
        print(f'NO POINTS FOR IMAGE {image_id}')
        continue
    points_3d_transformed = transform_points(np.array(points_3d_xyz)[:, :3], Tr)
    # Sanity check. Project points and get depths
    points_2d, colmap_depths = project_points(K, points_3d_transformed)
    # Sanity check. Filter points within the image boundaries
    mask = filter_points_in_image(points_2d, colmap_depths, image_width, image_height)
    if mask.any() == False:
        # This should never happen if the points from COLMAP are correct
        print(f'WARNING, ALL COLMAP POINTS SHOULD LIE INSIDE THE IMAGE PLANE {image_id}')
        continue
    # Recover metric depth from sensor data
    sensor_depth = np.array(Image.open(os.path.join(SENSOR_DEPTH_DIR, filename)), dtype=np.float64) / 1000.
    # Get the depth values using indexes (round to the 2d positions to the closest integer)
    points_2d = np.round(points_2d).astype(np.int32)
    points_2d = points_2d[:, [1, 0]]    # Change (u, v) for (v, u) to match the sensor_depth (height, width)
    # points_2d[:, 0] = points_2d[:, 0] - 6
    # points_2d[:, 1] = points_2d[:, 1] - 8
    # Check if out of bounds (the rgb images have already being cropped)
    mask = (points_2d[:, 0] >= 0) & (points_2d[:, 1] >= 0) & (points_2d[:, 0] < sensor_depth.shape[0]) & (points_2d[:, 1] < sensor_depth.shape[1])
    points_2d = points_2d[mask]
    metric_depths = sensor_depth[points_2d[:, 0], points_2d[:, 1]]
    mask_valid = metric_depths > 0
    metric_depths = metric_depths[mask_valid]
    
    print('Max colmap depths:', colmap_depths.max())
    if metric_depths.size > 0:
        print('Max metric depths:', metric_depths.max())
    print('NUM POINTS: ', metric_depths.size)
    # Accumulate the depths and errors
    ls_colmap_depths.extend(colmap_depths[mask][mask_valid])
    ls_sensor_depths.extend(metric_depths)
    ls_colmap_errors.extend(points_error[mask][mask_valid])
    ls_sensor_depths_all.extend(sensor_depth)
    print()

Image:  DSC01382
Max colmap depths: 13.304091228913187
Max metric depths: 2.434
NUM POINTS:  925

Image:  DSC01550
Max colmap depths: 10.301697996302547
Max metric depths: 1.895
NUM POINTS:  436

Image:  DSC01342
Max colmap depths: 8.498469144193656
Max metric depths: 1.444
NUM POINTS:  102

Image:  DSC01542
Max colmap depths: 10.892920136205415
Max metric depths: 2.031
NUM POINTS:  997

Image:  DSC01350
Max colmap depths: 13.241056859548555
Max metric depths: 1.9
NUM POINTS:  207

Image:  DSC01478
Max colmap depths: 12.891204914785547
Max metric depths: 2.224
NUM POINTS:  1630

Image:  DSC01294
Max colmap depths: 13.855130789950898
Max metric depths: 2.039
NUM POINTS:  203

Image:  DSC01526
Max colmap depths: 17.373540350219727
Max metric depths: 2.24
NUM POINTS:  352

Image:  DSC01278
Max colmap depths: 13.647001369905656
Max metric depths: 2.371
NUM POINTS:  1593

Image:  DSC01318
Max colmap depths: 13.560607445980224
Max metric depths: 1.589
NUM POINTS:  292

Image:  DSC01286
Max c

In [206]:
def compute_mean_scale(depths, errors, ground_truth_depths):
    depths = np.array(depths).squeeze()
    errors = np.array(errors).squeeze()
    ground_truth_depths = np.array(ground_truth_depths).squeeze()
    n = depths.shape[0]

    # Compute the mean scale weighting using the inverse of the errors
    weights = 1.0 / errors
    #scale = np.sum(weights * ground_truth_depths / depths) / np.sum(weights)  # Small difference using weights
    scale = np.mean(ground_truth_depths / depths)
    return scale

mean_scale = compute_mean_scale(ls_colmap_depths, ls_colmap_errors, ls_sensor_depths)

print('Mean scale: ', mean_scale)
print('Max depth: ', np.max(ls_sensor_depths))

Mean scale:  0.18446688255042698
Max depth:  2.434


In [207]:
# Write config.json file
config = {
    'name': os.path.basename(DATA_DIR),
    'max_depth': np.max([4.0, np.max(ls_sensor_depths)]),
    'mean_scale': mean_scale,
    "rgb_only": False
}
with open(os.path.join(DATA_DIR, CONFIG_FILENAME), 'w') as f:
    json.dump(config, f, indent=4)

In [1]:
# 
# barbara_depths = torch.load(os.path.join(DATA_DIR, 'depths.pth'))
# Get camera matrix
image_size, K = get_colmap_camera_matrix(os.path.join(COLMAP_DIR, 'cameras.txt'))
image_width, image_height = image_size
# Get dictionary of points
points_dict = get_colmap_dict_points(os.path.join(COLMAP_DIR, 'points3D.txt'))
ls_colmap_errors = []
ls_colmap_depths = []
ls_sensor_depths = []
ls_sensor_depths_all = []

# Iterate over the depths in the directory
for filename in os.listdir(SENSOR_DEPTH_DIR):
    #filename = os.path.basename(filename).split('.')[0] + '.npy'
    image_id = filename.split('.')[0]
    print('Image: ', image_id)
    # Get points from COLMAP for the same image
    colmap_images_path = os.path.join(COLMAP_DIR, 'images.txt')
    q, t, _, points_3d_ids = get_colmap_points(colmap_images_path, image_id)
    points_3d = [points_dict[point_id] for point_id in points_3d_ids]
    # Select the first three elements of the points_3d
    points_3d_xyz = np.array([point[:3] for point in points_3d])
    points_error = np.array([point[6] for point in points_3d])
    # Transform the 3D points into camera coordinates
    Tr = rotation_traslation_to_matrix(quaternion_to_rotation_matrix(q), t)
    if points_3d_xyz.shape[0] == 0:
        print(f'NO POINTS FOR IMAGE {image_id}')
        continue
    points_3d_transformed = transform_points(np.array(points_3d_xyz)[:, :3], Tr)
    # Sanity check. Project points and get depths
    points_2d, colmap_depths = project_points(K, points_3d_transformed)
    # Sanity check. Filter points within the image boundaries
    mask = filter_points_in_image(points_2d, colmap_depths, image_width, image_height)
    if mask.any() == False:
        # This should never happen if the points from COLMAP are correct
        print(f'WARNING, ALL COLMAP POINTS SHOULD LIE INSIDE THE IMAGE PLANE {image_id}')
        continue
    # Recover metric depth from sensor data
    sensor_depth = np.array(Image.open(os.path.join(SENSOR_DEPTH_DIR, filename)), dtype=np.float64) / 1000.
    # Get the depth values using indexes (round to the 2d positions to the closest integer)
    points_2d = np.round(points_2d).astype(np.int32)
    points_2d = points_2d[:, [1, 0]]    # Change (u, v) for (v, u) to match the sensor_depth (height, width)
    # points_2d[:, 0] = points_2d[:, 0] - 6
    # points_2d[:, 1] = points_2d[:, 1] - 8
    # Check if out of bounds (the rgb images have already being cropped)
    mask = (points_2d[:, 0] >= 0) & (points_2d[:, 1] >= 0) & (points_2d[:, 0] < sensor_depth.shape[0]) & (points_2d[:, 1] < sensor_depth.shape[1])
    points_2d = points_2d[mask]
    metric_depths = sensor_depth[points_2d[:, 0], points_2d[:, 1]]
    mask_valid = metric_depths > 0
    metric_depths = metric_depths[mask_valid]
    
    print('Max colmap depths:', colmap_depths.max())
    if metric_depths.size > 0:
        print('Max metric depths:', metric_depths.max())
    print('NUM POINTS: ', metric_depths.size)
    # Accumulate the depths and errors
    ls_colmap_depths.extend(colmap_depths[mask][mask_valid])
    ls_sensor_depths.extend(metric_depths)
    ls_colmap_errors.extend(points_error[mask][mask_valid])
    ls_sensor_depths_all.extend(sensor_depth)
    print()

NameError: name 'get_colmap_camera_matrix' is not defined